# Traditional Features

In [34]:
import nltk
nltk.download('punkt')
import pyphen
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize
from numpy import random
from tqdm import tqdm
import os
import ast

[nltk_data] Downloading package punkt to /home/soeren/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
def average_character_per_word(preprocessed_text):
    characters_per_wors = []
    for sentence in preprocessed_text:
        characters_per_wors += [len(word[0]) for word in sentence if word[1]!="SYM"]
    return np.mean(characters_per_wors)

test1 = [[["Ich", ""], ["studiere", ""], ["in", ""], ["Potsdam", ""], [".", "SYM"]]]
assert average_character_per_word(test1) == 5

In [36]:
def text_length(preprocessed_text):
    count = 0
    for sentence in preprocessed_text:
        count+= len(sentence)
    return count

In [37]:
def average_sentence_length(preprocessed_text):
    ''' Average length of a sentence. All characters and symbols are included.'''
    return np.mean([len(sentence) for sentence in preprocessed_text])

In [38]:
def average_syllable_word(preprocessed_text):
    '''Average length of a sentence. SYM are excluded and not counted'''
    pyphen.language_fallback('de_DE_variant1')
    dic = pyphen.Pyphen(lang='de_DE')
    counts = []
    for sentence in preprocessed_text:
        for word in sentence:
            if word[1]!="SYM":
                counts.append(syllable_count(word[0], dic))
    
    return np.mean(counts)

In [39]:
def syllable_count(string, dic):
    '''The number of syllables of a string is returned'''
    return len(dic.inserted(string).split("-"))

In [40]:
def Flesch_Reading_Ease_Score(preprocessed_text):
    ''' On a given text the score will produce a value between 1 and 100 where the higher
    the value the easier the text would be. Documents scoring 30 are very difficult to
    read while those scoring 70 should be easy to read.'''
    asl = average_sentence_length(preprocessed_text)
    asw = average_syllable_word(preprocessed_text)
    
    return 206.835 - (1.015 * asl) - (84.6 * asw)

In [41]:
def Flesch_Kincaid_readability(preprocessed_text):
    '''The Flesch-Kincaid readability formula (2.2) simplifies the Flesch score to produce a “grade
    level” which is easily interpretable (i.e., a text with a grade level of eight according to the formula
    could be thought appropriate for an eighth grader).'''
    asl = average_sentence_length(preprocessed_text)
    asw = average_syllable_word(preprocessed_text)
    
    return (0.4 * asl) + (12 * asw) - 15

In [42]:
def percentag_hard_words(preprocessed_text):
    pyphen.language_fallback('de_DE_variant1')
    dic = pyphen.Pyphen(lang='de_DE')
    count_hard_words=0
    length=0
    for sentence in preprocessed_text:
        for word in sentence:
            if word[1]!="SYM":
                length+=1
                if syllable_count(word[0], dic)>=3:
                    count_hard_words+=1
            
    return count_hard_words/length

In [43]:
def fog(preprocessed_text):
    hw = percentag_hard_words(preprocessed_text)
    asl = average_sentence_length(preprocessed_text)
                
    return 0.4 * (hw + asl)

In [44]:
def polysyllable_count(preprocessed_text):
    '''the number of words with 3 or more syllables in 30 sentences
    which shall be picked from the beginning, middle, and end of the document.'''
    pyphen.language_fallback('de_DE_variant1')
    dic = pyphen.Pyphen(lang='de_DE')
    part_len = int(len(preprocessed_text)/3)
    
    if part_len > 10:
        beginning_sentences = preprocessed_text[:part_len]
        middle_sentences = preprocessed_text[part_len:-part_len]
        end_sentences = preprocessed_text[-part_len:]
        picked_sentences = random.choice(beginning_sentences, 10) + random.choice(middle_sentences, 10) + random.choice(end_sentences, 10)
    else:
        picked_sentences = preprocessed_text
    
    polysyllable_count = 0
    for sentence in picked_sentences:
        for word in sentence:
            if syllable_count(word[0], dic)>=3:
                polysyllable_count+=1
    
    return polysyllable_count

In [45]:
def smog(preprocessed_text):
    psc = polysyllable_count(preprocessed_text)
    return 3 + np.sqrt(psc)

In [49]:
input_files = os.listdir("/home/soeren/Documents/TextDifficultyAssessmentGerman/datasets/RandomText")
input_files = [input_file for input_file in input_files if input_file.startswith("10_RandomText")]
def save_df(path,df):
    df.to_csv(path,sep="|",index=False)

for input_file in input_files:
    df=pd.read_csv("datasets/RandomText/" + input_file, sep="|", lineterminator='\n')

    df["preprocessedText"]  = [ast.literal_eval(text) for text in df["RFTagger"]]
    df["BF-average_sentence_length"] = [average_sentence_length(text) for text in tqdm(df["preprocessedText"])]
    df["BF-average_syllable_word"] = [average_syllable_word(text) for text in tqdm(df["preprocessedText"])]
    df["BF-Flesch_Reading_Ease_Score"] = [Flesch_Reading_Ease_Score(text) for text in tqdm(df["preprocessedText"])]
    df["BF-Flesch_Kincaid_readability"] = [Flesch_Kincaid_readability(text) for text in tqdm(df["preprocessedText"])]
    df["BF-percentag_hard_words"] = [percentag_hard_words(text) for text in tqdm(df["preprocessedText"])]
    df["BF-fog"] = [fog(text) for text in tqdm(df["preprocessedText"])]
    df["BF-polysyllable_count"] = [polysyllable_count(text) for text in tqdm(df["preprocessedText"])]
    df["BF-smog"] = [smog(text) for text in tqdm(df["preprocessedText"])]
    df["BF-average_character_per_word"] = [average_character_per_word(text) for text in tqdm(df["preprocessedText"])]
    df["BF-text_length"] = [text_length(text) for text in tqdm(df["preprocessedText"])]

    display(df.head())
    filename = "datasets/RandomText/FeatureSet1_BF/" + input_file
    save_df(filename,df)



  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 3211.57it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1032.23it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1074.45it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 923.72it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1476.87it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1274.61it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1563.48it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1370.09it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 2221.56it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 3024.74it/s]

,Text,Level,RFTagger,parsedText,preprocessedText,BF-average_sentence_length,BF-average_syllable_word,BF-Flesch_Reading_Ease_Score,BF-Flesch_Kincaid_readability,BF-percentag_hard_words,BF-fog,BF-polysyllable_count,BF-smog,BF-average_character_per_word,BF-text_length
0,Mir geht es gut. Viele Grüße Julia Wie gehasch...,A,"[[['Mir', 'PRO', 'Pers', 'Subst', '1', 'Dat', ...",['(ROOT (S (PPER Mir) (VVFIN geht) (PPER es) (...,"[[[Mir, PRO, Pers, Subst, 1, Dat, Sg, *], [geh...",5.333333,1.428571,80.564524,4.276190,0.000000,2.133333,0,3.00000,4.142857,16
1,Zweitens die Familie war gar nicht sympathisch...,B,"[[['Zweitens', 'ADV'], ['die', 'ART', 'Def', '...",['(ROOT (CS (CS (S (NP (ADV Zweitens) (ART die...,"[[[Zweitens, ADV], [die, ART, Def, Nom, Sg, Fe...",17.000000,1.613636,53.066364,11.163636,0.136364,6.854545,6,5.44949,5.181818,51
2,Wir kommen nochmals in der Beispiel der Mutter...,C,"[[['Wir', 'PRO', 'Pers', 'Subst', '1', 'Nom', ...",['(ROOT (S (PPER Wir) (VVFIN kommen) (ADV noch...,"[[[Wir, PRO, Pers, Subst, 1, Nom, Pl, *], [kom...",20.666667,1.965517,19.575575,16.852874,0.275862,8.377011,16,7.00000,5.948276,62




  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 3299.14it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1051.73it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1633.08it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1396.55it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1836.39it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1501.90it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1969.16it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1287.52it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1488.93it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 3110.73it/s]

,Text,Level,RFTagger,parsedText,preprocessedText,BF-average_sentence_length,BF-average_syllable_word,BF-Flesch_Reading_Ease_Score,BF-Flesch_Kincaid_readability,BF-percentag_hard_words,BF-fog,BF-polysyllable_count,BF-smog,BF-average_character_per_word,BF-text_length
0,Die Stadt habe viele zur Nachtunterhaltung . M...,A,"[[['Die', 'ART', 'Def', 'Nom', 'Sg', 'Fem'], [...",['(ROOT (S (NP (ART Die) (NN Stadt)) (VAFIN ha...,"[[[Die, ART, Def, Nom, Sg, Fem], [Stadt, N, Re...",6.333333,1.941176,36.183137,10.827451,0.176471,2.603922,3,4.732051,6.117647,19
1,"Es geht immer wieder besser. * Aber, ist diese...",B,"[[['Es', 'PRO', 'Pers', 'Subst', '3', 'Nom', '...",['(ROOT (S (PPER Es) (VVFIN geht) (AP (AVP (AD...,"[[[Es, PRO, Pers, Subst, 3, Nom, Sg, Neut], [g...",5.333333,1.500000,74.521667,5.133333,0.000000,2.133333,0,3.000000,4.750000,16
2,Trotzdem werden jungen britischen Entrepreneur...,C,"[[['Trotzdem', 'PROADV', 'Dem'], ['werden', 'V...",['(ROOT (S (PROAV Trotzdem) (VAFIN werden) (NP...,"[[[Trotzdem, PROADV, Dem], [werden, VFIN, Aux,...",10.333333,1.703704,52.213333,9.577778,0.222222,4.222222,6,5.449490,5.592593,31




  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 3137.10it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 923.18it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 557.48it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 742.84it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 879.12it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 874.60it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 310.97it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 932.69it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 2043.68it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1931.97it/s]

...

,Text,Level,RFTagger,parsedText,preprocessedText,BF-average_sentence_length,BF-average_syllable_word,BF-Flesch_Reading_Ease_Score,BF-Flesch_Kincaid_readability,BF-percentag_hard_words,BF-fog,BF-polysyllable_count,BF-smog,BF-average_character_per_word,BF-text_length
0,Die Fahrkarte war für das Flugzeugreise zum St...,A,"[[['Die', 'ART', 'Def', 'Nom', 'Sg', 'Fem'], [...",['(ROOT (S (NP (ART Die) (NN Fahrkarte)) (VAFI...,"[[[Die, ART, Def, Nom, Sg, Fem], [Fahrkarte, N...",7.5,1.461538,75.576346,5.538462,0.153846,3.061538,2,4.414214,4.846154,15
1,"Folglich, finden die Deutschen diese Idee nich...",B,"[[['Folglich', 'ADV'], [',', 'SYM', 'Pun', 'Co...","['(ROOT (CS (S (ADV Folglich) ($, ,) (VVFIN fi...","[[[Folglich, ADV], [,, SYM, Pun, Comma], [find...",26.5,1.680851,37.737500,15.770213,0.191489,10.676596,9,6.000000,5.510638,53
2,Die Vereinigten-Staaten können sehr stolz sein...,C,"[[['Die', 'ART', 'Def', 'Nom', 'Pl', 'Masc'], ...",['(ROOT (S (NP (ART Die) (NN Vereinigten-Staat...,"[[[Die, ART, Def, Nom, Pl, Masc], [Vereinigten...",16.5,1.678571,48.080357,11.742857,0.107143,6.642857,3,4.732051,5.357143,33




  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1581.36it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 664.22it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 547.99it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 405.77it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 808.88it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 773.24it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 394.66it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 761.40it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 2171.71it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 4331.47it/s]

,Text,Level,RFTagger,parsedText,preprocessedText,BF-average_sentence_length,BF-average_syllable_word,BF-Flesch_Reading_Ease_Score,BF-Flesch_Kincaid_readability,BF-percentag_hard_words,BF-fog,BF-polysyllable_count,BF-smog,BF-average_character_per_word,BF-text_length
0,Demoliert stach Harry Eltern ab . Ich möchte z...,A,"[[['Demoliert', 'N', 'Name', 'Nom', 'Sg', '*']...",['(ROOT (S (VVPP Demoliert) (VVFIN stach) (NP ...,"[[[Demoliert, N, Name, Nom, Sg, *], [stach, VF...",8.666667,1.617647,61.185392,7.878431,0.132353,3.519608,9,6.000000,4.985294,78
1,"Der Artikel in die Presse ist viel kürzer, ohn...",B,"[[['Der', 'ART', 'Def', 'Nom', 'Sg', 'Masc'], ...",['(ROOT (S (NP (ART Der) (NN Artikel) (PP (APP...,"[[[Der, ART, Def, Nom, Sg, Masc], [Artikel, N,...",15.555556,1.809917,37.927103,12.941230,0.223140,6.311478,27,8.196152,5.338843,140
2,"Leute sagen auch, dass die Mitarbeiten des Ver...",C,"[[['Leute', 'N', 'Reg', 'Nom', 'Pl', '*'], ['s...",['(ROOT (S (NN Leute) (VVFIN sagen) (ADV auch)...,"[[[Leute, N, Reg, Nom, Pl, *], [sagen, VFIN, F...",17.888889,1.993103,20.061226,16.072797,0.262069,7.260383,38,9.164414,6.041379,161




  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 3350.98it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1417.79it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 643.86it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1352.27it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 430.51it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 607.02it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 875.39it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 893.17it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 1977.51it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 3339.41it/s]

,Text,Level,RFTagger,parsedText,preprocessedText,BF-average_sentence_length,BF-average_syllable_word,BF-Flesch_Reading_Ease_Score,BF-Flesch_Kincaid_readability,BF-percentag_hard_words,BF-fog,BF-polysyllable_count,BF-smog,BF-average_character_per_word,BF-text_length
0,Die Berichte sollte interessant sein . Ich mus...,A,"[[['Die', 'ART', 'Def', 'Nom', 'Pl', 'Masc'], ...",['(ROOT (S (NP (ART Die) (NN Berichte)) (VMFIN...,"[[[Die, ART, Def, Nom, Pl, Masc], [Berichte, N...",8.250,1.525424,69.410403,6.605085,0.135593,3.354237,8,5.828427,4.644068,66
1,Damit kann man auch investieren. Die Situation...,B,"[[['Damit', 'PROADV', 'Dem'], ['kann', 'VFIN',...",['(ROOT (S (PROAV Damit) (VMFIN kann) (PIS man...,"[[[Damit, PROADV, Dem], [kann, VFIN, Mod, 3, S...",13.375,1.842105,37.417270,12.455263,0.210526,5.434211,20,7.472136,5.494737,107
2,"Genau das , was ich oben beschrieben habe , wü...",C,"[[['Genau', 'ADV'], ['das', 'PRO', 'Dem', 'Sub...","['(ROOT (S (NP (ADV Genau) (PDS das)) ($, ,) (...","[[[Genau, ADV], [das, PRO, Dem, Subst, -, Nom,...",15.125,1.796117,39.531669,12.603398,0.194175,6.127670,20,7.472136,5.446602,121
